In [1]:
import pandas as pd
#series = pd.read_csv('/home/cloudera/FeedDB.csv',sep=',',header=None,index_col=0).sort_index()
import warnings
warnings.filterwarnings("ignore")

In [2]:
#x_data = []
#y_data = []
 
#for d in range(1,df.shape[0]):
#    x = df.iloc[d-1:d].values.ravel()
#    y = df.iloc[d].values[0]
#    x_data.append(x)
#    y_data.append(y)

In [3]:
#df.head(3)
#x_data
#y_data

In [4]:
from pandas import Series
from matplotlib import pyplot  as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [5]:
series = Series.from_csv('/home/cloudera/FeedDB_backup.csv',header=None,index_col=0,parse_dates=[0])
series.index
series.shape[0]
print(series.describe())
plt.plot(series)
#plt.xticks(rotation='vertical')
plt.show()
from pandas.tools.plotting import autocorrelation_plot

autocorrelation_plot(series)
plt.show()

In [6]:
series = Series.from_csv('/home/cloudera/FeedDB_backup.csv',header=None,index_col=0,parse_dates=[0])
series.index
series.shape[0]
print(series.describe())
plt.plot(series)
#plt.xticks(rotation='vertical')
plt.show()
from pandas.tools.plotting import autocorrelation_plot

autocorrelation_plot(series)
plt.show()

In [7]:
#transform here for simplicity

In [8]:
import numpy as np
rcParams['figure.figsize'] = 15, 6
ts_log = np.log(series)
#ts_log.plot()
plt.plot(ts_log)
plt.xticks(rotation='vertical')
plt.show()
ts_log.head(3)

In [9]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries,window_size):
    plt.xticks(rotation='vertical')
    #Determing rolling statistics
    rolmean = pd.rolling_mean(timeseries, window=window_size)
    rolstd = pd.rolling_std(timeseries, window=window_size)
    plt.show(block=False)
    
    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean/Avg')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [10]:
#series.head(3)
test_stationarity(ts_log,7)

In [11]:
#exponentially weighted moving average

In [12]:
exp_weighted_avg = pd.ewma(ts_log, halflife= 5)
plt.plot(ts_log)
plt.plot(exp_weighted_avg,color='red')
plt.show()


In [13]:
ts_log_ewma_diff = ts_log - exp_weighted_avg
test_stationarity(ts_log_ewma_diff,7)

In [14]:
# decomposing


In [15]:
# no use
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(ts_log,freq=30)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(ts_log, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

plt.show()

In [16]:
ts_log_decompose = residual
ts_log_decompose.dropna(inplace=True)
test_stationarity(ts_log_decompose,7)

In [17]:
ts_log_diff = ts_log - ts_log.shift()
plt.plot(ts_log_diff)
plt.show()

In [18]:
'''ARIMA. I won’t go into the technical details but you should understand these concepts in detail if you wish to apply them more effectively. ARIMA stands for Auto-Regressive Integrated Moving Averages. The ARIMA forecasting for a stationary time series is nothing but a linear (like a linear regression) equation. The predictors depend on the parameters (p,d,q) of the ARIMA model:

Number of AR (Auto-Regressive) terms (p): AR terms are just lags of dependent variable. For instance if p is 5, the predictors for x(t) will be x(t-1)….x(t-5).
Number of MA (Moving Average) terms (q): MA terms are lagged forecast errors in prediction equation. For instance if q is 5, the predictors for x(t) will be e(t-1)….e(t-5) where e(i) is the difference between the moving average at ith instant and actual value.
Number of Differences (d): These are the number of nonseasonal differences, i.e. in this case we took the first order difference. So either we can pass that variable and put d=0 or pass the original variable and put d=1. Both will generate same results.

An importance concern here is how to determine the value of ‘p’ and ‘q’. We use two plots to determine these numbers. Lets discuss them first.

Autocorrelation Function (ACF): It is a measure of the correlation between the the TS with a lagged version of itself. For instance at lag 5, ACF would compare series at time instant ‘t1’…’t2’ with series at instant ‘t1-5’…’t2-5’ (t1-5 and t2 being end points).
Partial Autocorrelation Function (PACF): This measures the correlation between the TS with a lagged version of itself but after eliminating the variations already explained by the intervening comparisons. Eg at lag 5, it will check the correlation but remove the effects already explained by lags 1 to 4.

The ACF and PACF plots for the TS after differencing can be plotted as:
'''

In [19]:
#ACF and PACF plots:
from statsmodels.tsa.stattools import acf, pacf

In [20]:
lag_acf = acf(series, nlags=7)
lag_pacf = pacf(ts_log_diff, nlags=7, method='ols')

In [21]:
plt.plot(lag_acf)
plt.show()

In [22]:
plt.plot(lag_pacf)
plt.show()